In [1]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show langchain

Name: langchain
Version: 0.3.19
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: C:\Users\sivak\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: jupyter_ai_magics, langchain-community
Note: you may need to restart the kernel to use updated packages.


### Python.env

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys from .env
load_dotenv(find_dotenv(), override=True)

#os.environ.get('OPENAI_API_KEY')

True

In [4]:
# loading PDF, DOCX and TXT files as LangChain Documents
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data

In [19]:
# wikipedia
def load_from_wikipedia(query, lang='en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [7]:
data = load_document('files/us_constitution.pdf')
#print(data[1].page_content)
# print(data[10].metadata)

print(f'You have {len(data)} pages in your data')
print(f'There are {len(data[20].page_content)} characters in the page')

Loading files/us_constitution.pdf
You have 41 pages in your data
There are 1173 characters in the page


In [26]:
#data = load_document('files/the_great_gatsby.docx')
#print(data[0].page_content)

In [21]:
pip install wikipedia

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=da4d21508b85b78ce31cee51ca50d43ee2909c52b696f4a1bc95b918fea40fb8
  Stored in directory: c:\users\sivak\appdata\local\pip\cache\wheels\79\1d\c8\b64e19423cc5a2a339450ea5d145e7c8eb3d4aa2b150cde33b
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [23]:
#data = load_from_wikipedia('GPT-4', 'de')
#print(data[0].page_content)

OpenAI, Inc. ist ein US-amerikanisches nicht-börsennotiertes Softwareunternehmen, das sich seit Ende 2015 mit der Erforschung von künstlicher Intelligenz (KI, englisch Artificial Intelligence, AI) beschäftigt. Anfänglich war das Ziel von OpenAI, künstliche Intelligenz auf Open-Source-Basis zu entwickeln. Das Unternehmen wurde vorerst als Non-Profit geführt. 2019 wurde die gewinnorientierte Tochtergesellschaft OpenAI Global, LLC gegründet, in der Microsoft größter Investor ist. OpenAI ist vor allem bekannt für die Entwicklung der generativen vortrainierten Transformer (GPT) – auch generative künstliche Intelligenz, kurz GenAI, bezeichnet – und der daraus abgeleiteten Softwareprodukte wie ChatGPT oder DALL-E.


== Geschichte ==


=== Gründungsphase und Mission ===
Der Gründung von OpenAI im Jahr 2015 ging bereits eine lange Debatte um die Risiken von KI voraus. Die Wissenschaftler Stephen Hawking und Stuart Jonathan Russell etwa hatten Befürchtungen geäußert, wenn künstliche Intelligenz 

### Chunking Data

In [14]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks

In [15]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    # check prices here: https://openai.com/pricing
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}')

In [16]:
chunks = chunk_data(data)
print(len(chunks))
print(chunks[10].page_content)

224
Maryland six, V irginia ten, North Carolina five, South Carolina five, and 
 Georgia three. 
 When vacancies happen in the Representation from any State, the 
 Executive Authority thereof shall issue W rits of Election to fill such 
 V acancies.


In [17]:
print_embedding_cost(chunks)

Total Tokens: 9842
Embedding Cost in USD: 0.000197


### Embedding and Uploading to a Vector Database (Pinecone)

In [18]:
def insert_or_fetch_embeddings(index_name, chunks):
    # importing the necessary libraries and initializing the Pinecone client
    import pinecone
    from langchain_community.vectorstores import Pinecone
    from langchain_openai import OpenAIEmbeddings
    from pinecone import ServerlessSpec

    
    pc = pinecone.Pinecone()
        
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)  # 512 works as well

    # loading from existing index
    if index_name in pc.list_indexes().names():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        # creating the index and embedding the chunks into the index 
        print(f'Creating index {index_name} and embeddings ...', end='')

        # creating a new index
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric='cosine',
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
        ) 
        )

        # processing the input documents, generating embeddings using the provided `OpenAIEmbeddings` instance,
        # inserting the embeddings into the index and returning a new Pinecone vector store object. 
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')
        
    return vector_store

In [19]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pc = pinecone.Pinecone()
    
    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes ... ')
        for index in indexes:
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')

In [20]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


In [21]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name=index_name, chunks=chunks)

Creating index askadocument and embeddings ...Ok


### Asking and Getting Answers

In [22]:
def ask_and_get_answer(vector_store, q, k=3):
    from langchain.chains import RetrievalQA
    from langchain_openai import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.invoke(q)
    return answer

In [25]:
q = 'What about The House of Representatives.translate in japanees'
answer = ask_and_get_answer(vector_store, q)
print(answer)

{'query': 'What about The House of Representatives.translate in japanees', 'result': '衆議院'}


### While Loop for Asking Questions

In [58]:
import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break
    
    answer = ask_and_get_answer(vector_store, q)
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')


Write Quit or Exit to quit.


Question #1:  what is bill of rights



Answer: {'query': 'what is bill of rights', 'result': 'The Bill of Rights refers to the first ten amendments to the United States Constitution. These amendments were added to the Constitution in 1791 to guarantee specific rights and freedoms to the American people. The Bill of Rights includes protections such as freedom of speech, religion, and the right to bear arms.'}

 -------------------------------------------------- 



Question #2:  quit


Quitting ... bye bye!
